<a href="https://colab.research.google.com/github/Balaji-A-2823/Machine_Learning_By_Providing_Dataset/blob/main/ann_reg_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Neural Network Power Generation Analysis - Quick Start Guide
# This script demonstrates the ANN implementation converted from SVM

import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')


def quick_ann_analysis(csv_file='/content/ann_project/MW_Daily.csv'):
    """
    Quick ANN analysis for power generation data

    This function demonstrates:
    1. Data loading and preprocessing
    2. ANN regression for total power prediction
    3. ANN classification for demand level prediction
    4. Model evaluation and predictions
    """

    print("="*60)
    print("ANN POWER GENERATION ANALYSIS - QUICK START")
    print("="*60)

    # Step 1: Load and preprocess data
    print("1. Loading and preprocessing data...")

    try:
        # Try to load the actual CSV file
        df = pd.read_csv(csv_file)
        df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
        print(f"   Loaded dataset from: {csv_file}")
    except FileNotFoundError:
        # Create sample dataset if file not found
        print("   CSV file not found. Creating sample data for demonstration...")
        np.random.seed(42)
        n_samples = 1000

        dates = pd.date_range(start='2020-01-01', periods=n_samples, freq='D')

        df = pd.DataFrame({
            'date': dates,
            'n_mw': np.random.normal(50000, 8000, n_samples),
            'w_mw': np.random.normal(47000, 7000, n_samples),
            's_mw': np.random.normal(40000, 6000, n_samples),
            'e_mw': np.random.normal(20000, 4000, n_samples),
            'ne_mw': np.random.normal(2500, 500, n_samples),
            'year': dates.year,
        })

        # Create ind_mw as sum with some noise
        df['ind_mw'] = (df['n_mw'] + df['w_mw'] + df['s_mw'] +
                        df['e_mw'] + df['ne_mw'] +
                        np.random.normal(0, 5000, n_samples))

    print(f"   Dataset shape: {df.shape}")
    print(f"   Date range: {df['date'].iloc[0].strftime('%d-%m-%Y')} to {df['date'].iloc[-1].strftime('%d-%m-%Y')}")

    # Create additional features
    df['month_num'] = df['date'].dt.month
    df['weekday'] = df['date'].dt.weekday
    df['quarter'] = df['date'].dt.quarter

    # Create demand categories
    q1, q2 = df['ind_mw'].quantile([0.33, 0.67])
    conditions = [
        df['ind_mw'] <= q1,
        (df['ind_mw'] > q1) & (df['ind_mw'] <= q2),
        df['ind_mw'] > q2
    ]
    choices = ['Low', 'Medium', 'High']
    df['demand_class'] = np.select(conditions, choices, default='Unknown')

    print(f"   Created {len(df.columns)} features")
    print(f"   Demand distribution: {df['demand_class'].value_counts().to_dict()}")

    # Step 2: Prepare features
    print("2. Preparing features for Neural Networks...")

    features = ['n_mw', 'w_mw', 's_mw', 'e_mw', 'ne_mw',
                'year', 'month_num', 'weekday', 'quarter']

    X = df[features]
    y_regression = df['ind_mw']  # Total power (continuous)
    y_classification = df['demand_class']  # Demand level (categorical)

    print(f"   Using {len(features)} features: {features}")

    # Step 3: Train ANN Regression Model
    print("3. Training ANN Regression model...")

    # Split data for regression
    X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
        X, y_regression, test_size=0.2, random_state=42
    )

    # Scale features (very important for neural networks)
    scaler_reg = StandardScaler()
    X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
    X_test_reg_scaled = scaler_reg.transform(X_test_reg)

    # Train ANN regression model
    # MLPRegressor parameters explained:
    # - hidden_layer_sizes: Architecture of hidden layers
    # - activation: Activation function for hidden layers
    # - solver: Algorithm for weight optimization
    # - alpha: L2 penalty parameter for regularization
    # - max_iter: Maximum iterations for convergence

    ann_regressor = MLPRegressor(
        hidden_layer_sizes=(64, 32),   # Two hidden layers with 64 and 32 neurons
        activation='relu',             # ReLU activation function
        solver='lbfgs',               # LBFGS optimizer (good for smaller datasets)
        alpha=0.01,                   # L2 regularization
        max_iter=2000,                # Maximum iterations
        random_state=42
    )

    ann_regressor.fit(X_train_reg_scaled, y_train_reg)

    # Evaluate regression
    y_pred_reg = ann_regressor.predict(X_test_reg_scaled)
    r2_reg = r2_score(y_test_reg, y_pred_reg)
    rmse_reg = np.sqrt(mean_squared_error(y_test_reg, y_pred_reg))
    mae_reg = np.mean(np.abs(y_test_reg - y_pred_reg))

    print(f"   ANN Regression Results:")
    print(f"   - Architecture: {ann_regressor.hidden_layer_sizes}")
    print(f"   - Training iterations: {ann_regressor.n_iter_}")
    print(f"   - R² Score: {r2_reg:.4f} ({r2_reg*100:.1f}% of variance explained)")
    print(f"   - RMSE: {rmse_reg:.0f} MW")
    print(f"   - MAE: {mae_reg:.0f} MW")

    # Step 4: Train ANN Classification Model
    print("4. Training ANN Classification model...")

    # Encode categorical labels
    label_encoder = LabelEncoder()
    y_classification_encoded = label_encoder.fit_transform(y_classification)

    # Split data for classification
    X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
        X, y_classification_encoded, test_size=0.2, random_state=42,
        stratify=y_classification_encoded
    )

    # Scale features for classification
    scaler_clf = StandardScaler()
    X_train_clf_scaled = scaler_clf.fit_transform(X_train_clf)
    X_test_clf_scaled = scaler_clf.transform(X_test_clf)

    # Train ANN classification model
    ann_classifier = MLPClassifier(
        hidden_layer_sizes=(64, 32),   # Two hidden layers
        activation='relu',             # ReLU activation
        solver='adam',                 # Adam optimizer
        alpha=0.01,                   # L2 regularization
        max_iter=500,                 # Maximum iterations
        random_state=42
    )

    ann_classifier.fit(X_train_clf_scaled, y_train_clf)

    # Evaluate classification
    y_pred_clf = ann_classifier.predict(X_test_clf_scaled)
    accuracy_clf = accuracy_score(y_test_clf, y_pred_clf)

    # Convert back to original labels for reporting
    y_pred_clf_labels = label_encoder.inverse_transform(y_pred_clf)
    y_test_clf_labels = label_encoder.inverse_transform(y_test_clf)

    print(f"   ANN Classification Results:")
    print(f"   - Architecture: {ann_classifier.hidden_layer_sizes}")
    print(f"   - Training iterations: {ann_classifier.n_iter_}")
    print(f"   - Accuracy: {accuracy_clf:.4f} ({accuracy_clf*100:.1f}%)")
    print(f"   - Classes: {list(label_encoder.classes_)}")

    # Step 5: Make Predictions on Sample Data
    print("5. Making predictions on sample data...")

    # Create sample data
    sample_data = pd.DataFrame({
        'n_mw': [45000, 55000, 40000],
        'w_mw': [46000, 48000, 42000],
        's_mw': [38000, 41000, 35000],
        'e_mw': [19000, 21000, 17000],
        'ne_mw': [2500, 2800, 2200],
        'year': [2021, 2021, 2021],
        'month_num': [6, 6, 12],
        'weekday': [1, 3, 5],
        'quarter': [2, 2, 4]
    })

    # Scale sample data
    sample_scaled_reg = scaler_reg.transform(sample_data[features])
    sample_scaled_clf = scaler_clf.transform(sample_data[features])

    # Make predictions
    total_power_pred = ann_regressor.predict(sample_scaled_reg)
    demand_level_pred_enc = ann_classifier.predict(sample_scaled_clf)
    demand_level_pred = label_encoder.inverse_transform(demand_level_pred_enc)

    print("   Sample Predictions:")
    for i in range(len(sample_data)):
        regional_sum = (sample_data.iloc[i]['n_mw'] + sample_data.iloc[i]['w_mw'] +
                       sample_data.iloc[i]['s_mw'] + sample_data.iloc[i]['e_mw'] +
                       sample_data.iloc[i]['ne_mw'])

        print(f"   Sample {i+1}:")
        print(f"     Regional Power: {regional_sum:,} MW")
        print(f"     Predicted Total: {total_power_pred[i]:,.0f} MW")
        print(f"     Predicted Level: {demand_level_pred[i]}")
        print()

    # Step 6: Summary
    print("6. Summary:")
    print("   ✓ ANN Regression model successfully predicts total power generation")
    print("   ✓ ANN Classification model accurately categorizes demand levels")
    print("   ✓ Neural networks show good performance with proper tuning")

    # Return trained models for further use
    models = {
        'regression': {
            'model': ann_regressor,
            'scaler': scaler_reg,
            'features': features,
            'metrics': {'r2': r2_reg, 'rmse': rmse_reg, 'mae': mae_reg}
        },
        'classification': {
            'model': ann_classifier,
            'scaler': scaler_clf,
            'label_encoder': label_encoder,
            'features': features,
            'metrics': {'accuracy': accuracy_clf}
        }
    }

    print("\n" + "="*60)
    print("ANN ANALYSIS COMPLETED SUCCESSFULLY!")
    print("="*60)

    return models


def predict_new_data_ann(models, new_data):
    """
    Use trained ANN models to predict on new data

    Parameters:
    - models: Dictionary with trained models from quick_ann_analysis()
    - new_data: DataFrame with required feature columns

    Returns:
    - DataFrame with predictions
    """

    features = models['regression']['features']

    # Regression prediction
    reg_scaler = models['regression']['scaler']
    reg_model = models['regression']['model']
    X_reg_scaled = reg_scaler.transform(new_data[features])
    total_power_pred = reg_model.predict(X_reg_scaled)

    # Classification prediction
    clf_scaler = models['classification']['scaler']
    clf_model = models['classification']['model']
    label_encoder = models['classification']['label_encoder']
    X_clf_scaled = clf_scaler.transform(new_data[features])
    demand_level_pred_enc = clf_model.predict(X_clf_scaled)
    demand_level_pred = label_encoder.inverse_transform(demand_level_pred_enc)

    # Combine results
    predictions = pd.DataFrame({
        'predicted_total_power': total_power_pred,
        'predicted_demand_level': demand_level_pred
    })

    return predictions


def compare_ann_architectures(X, y_reg, y_clf, features):
    """
    Compare different ANN architectures and hyperparameters

    Parameters:
    - X, y_reg, y_clf: Training data
    - features: Feature names

    Returns:
    - Dictionary with comparison results
    """

    print("\n" + "="*60)
    print("COMPARING DIFFERENT ANN ARCHITECTURES")
    print("="*60)

    # Different architectures to test
    architectures = [
        (32,),           # Single hidden layer
        (64, 32),        # Two hidden layers
        (100, 50, 25),   # Three hidden layers
        (128, 64, 32)    # Deeper network
    ]

    # Different solvers to test
    solvers = ['lbfgs', 'adam', 'sgd']

    results = []

    # Encode classification labels
    label_encoder = LabelEncoder()
    y_clf_encoded = label_encoder.fit_transform(y_clf)

    # Split data
    X_train, X_test, y_reg_train, y_reg_test = train_test_split(
        X, y_reg, test_size=0.2, random_state=42
    )
    _, _, y_clf_train, y_clf_test = train_test_split(
        X, y_clf_encoded, test_size=0.2, random_state=42, stratify=y_clf_encoded
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    for arch in architectures:
        for solver in solvers:
            try:
                # Test regression
                reg_model = MLPRegressor(
                    hidden_layer_sizes=arch,
                    solver=solver,
                    alpha=0.01,
                    max_iter=500,
                    random_state=42
                )
                reg_model.fit(X_train_scaled, y_reg_train)
                reg_pred = reg_model.predict(X_test_scaled)
                reg_r2 = r2_score(y_reg_test, reg_pred)

                # Test classification
                clf_model = MLPClassifier(
                    hidden_layer_sizes=arch,
                    solver=solver,
                    alpha=0.01,
                    max_iter=500,
                    random_state=42
                )
                clf_model.fit(X_train_scaled, y_clf_train)
                clf_pred = clf_model.predict(X_test_scaled)
                clf_accuracy = accuracy_score(y_clf_test, clf_pred)

                results.append({
                    'architecture': arch,
                    'solver': solver,
                    'regression_r2': reg_r2,
                    'classification_accuracy': clf_accuracy,
                    'reg_iterations': reg_model.n_iter_,
                    'clf_iterations': clf_model.n_iter_
                })

                print(f"   {arch} + {solver}: R²={reg_r2:.3f}, Accuracy={clf_accuracy:.3f}")

            except Exception as e:
                print(f"   {arch} + {solver}: Failed ({str(e)[:50]}...)")

    # Find best configurations
    best_reg = max(results, key=lambda x: x['regression_r2'])
    best_clf = max(results, key=lambda x: x['classification_accuracy'])

    print(f"\nBest Regression: {best_reg['architecture']} + {best_reg['solver']} (R²={best_reg['regression_r2']:.3f})")
    print(f"Best Classification: {best_clf['architecture']} + {best_clf['solver']} (Acc={best_clf['classification_accuracy']:.3f})")

    return results


# Example usage
if __name__ == "__main__":
    try:
        # Run the quick analysis
        print("Starting ANN Power Generation Analysis...")
        trained_models = quick_ann_analysis("/content/ann_project/MW_Daily.csv")

        if trained_models:
            # Test with new scenarios
            new_scenarios = pd.DataFrame({
                'n_mw': [50000, 60000, 35000],
                'w_mw': [48000, 52000, 38000],
                's_mw': [42000, 46000, 32000],
                'e_mw': [20000, 24000, 15000],
                'ne_mw': [2600, 2900, 2000],
                'year': [2024, 2024, 2024],
                'month_num': [7, 8, 3],
                'weekday': [0, 2, 4],
                'quarter': [3, 3, 1]
            })

            predictions = predict_new_data_ann(trained_models, new_scenarios)

            print("\nNew Scenario Predictions:")
            for i in range(len(new_scenarios)):
                print(f"Scenario {i+1}: Total Power = {predictions.iloc[i]['predicted_total_power']:,.0f} MW, "
                      f"Level = {predictions.iloc[i]['predicted_demand_level']}")

        # Display key differences from SVM
        print("\n" + "="*60)
        print("KEY DIFFERENCES FROM SVM TO ANN")
        print("="*60)

        print("\n1. Model Architecture:")
        print("   - SVM: Single layer with kernel tricks")
        print("   - ANN: Multi-layer perceptron with hidden layers")

        print("\n2. Hyperparameters:")
        print("   - SVM: C (regularization), kernel, gamma")
        print("   - ANN: hidden_layer_sizes, activation, alpha, solver")

        print("\n3. Training Process:")
        print("   - SVM: Optimization problem solving")
        print("   - ANN: Iterative gradient descent")

        print("\n4. Scalability:")
        print("   - SVM: Better for smaller datasets")
        print("   - ANN: Better for larger datasets, more flexible")

        print("\n5. Feature Scaling:")
        print("   - Both require feature scaling, but ANN is more sensitive")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please ensure the data file exists or the script will use sample data.")

# Additional ANN-specific tips and best practices
print("\n" + "="*60)
print("ANN BEST PRACTICES FOR POWER GENERATION ANALYSIS")
print("="*60)

print("\n1. Network Architecture:")
print("   - Start with 1-2 hidden layers")
print("   - Use 64-128 neurons per layer for power data")
print("   - Avoid very deep networks for tabular data")

print("\n2. Activation Functions:")
print("   - ReLU: Good default choice")
print("   - Tanh: For data with negative values")
print("   - Sigmoid: For binary classification only")

print("\n3. Solvers:")
print("   - LBFGS: Best for small datasets (<1000 samples)")
print("   - Adam: Good for larger datasets")
print("   - SGD: Use with learning rate scheduling")

print("\n4. Regularization:")
print("   - Alpha: 0.001-0.1 (start with 0.01)")
print("   - Early stopping: Prevent overfitting")
print("   - Dropout: Can be added with custom implementation")

print("\n5. Data Preprocessing:")
print("   - Feature scaling is mandatory")
print("   - Handle missing values carefully")
print("   - Consider feature engineering")

Starting ANN Power Generation Analysis...
ANN POWER GENERATION ANALYSIS - QUICK START
1. Loading and preprocessing data...
   Loaded dataset from: /content/ann_project/MW_Daily.csv
   Dataset shape: (2742, 10)
   Date range: 01-01-2014 to 04-07-2021
   Created 14 features
   Demand distribution: {'Medium': 932, 'Low': 905, 'High': 905}
2. Preparing features for Neural Networks...
   Using 9 features: ['n_mw', 'w_mw', 's_mw', 'e_mw', 'ne_mw', 'year', 'month_num', 'weekday', 'quarter']
3. Training ANN Regression model...
   ANN Regression Results:
   - Architecture: (64, 32)
   - Training iterations: 2000
   - R² Score: 1.0000 (100.0% of variance explained)
   - RMSE: 16 MW
   - MAE: 10 MW
4. Training ANN Classification model...
   ANN Classification Results:
   - Architecture: (64, 32)
   - Training iterations: 193
   - Accuracy: 0.9854 (98.5%)
   - Classes: ['High', 'Low', 'Medium']
5. Making predictions on sample data...
   Sample Predictions:
   Sample 1:
     Regional Power: 150,500